In [2]:
# !pip install yfpy

In [3]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from yfpy.query import YahooFantasySportsQuery
from yahoo_oauth import OAuth2
import xml.etree.ElementTree as ET


In [4]:
load_dotenv()
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

In [5]:
def strip_namespace(tag):
    """Remove namespace from the tag name."""
    return tag.split("}")[-1] if "}" in tag else tag

def parse_element(element):
    parsed_data = {}
    children = list(element)

    # Check if all children have the same tag (implying a list)
    child_tags = [strip_namespace(child.tag) for child in children]
    is_list = len(set(child_tags)) == 1 and len(children) > 1

    # Convert child elements into dictionary keys
    if is_list:
        return [parse_element(child) for child in children]  # Return a list directly
    else:
        for child in children:
            tag = strip_namespace(child.tag)
            child_data = parse_element(child)

            # Handle multiple children with the same tag by storing them as lists
            if tag in parsed_data:
                if not isinstance(parsed_data[tag], list):
                    parsed_data[tag] = [parsed_data[tag]]
                parsed_data[tag].append(child_data)
            else:
                parsed_data[tag] = child_data

    # If the element has text content, add it
    text = element.text.strip() if element.text else ""
    if text and not parsed_data:
        return text  # Return text if no nested structure

    return parsed_data or text  # Return text if no nested structure

def xml_to_dict(xml_string):
    root = ET.fromstring(xml_string)
    return {strip_namespace(root.tag): parse_element(root)}

In [6]:
# Use last season stats so there aren't any spoilers
current = False
league_id = "67269" if current else "97108"
game_id = 453 if current else 427

query = YahooFantasySportsQuery(
    league_id=league_id,
    game_code="nhl",
    game_id=game_id,
    yahoo_consumer_key=CLIENT_ID,
    yahoo_consumer_secret=CLIENT_SECRET
)

oauth = OAuth2(
    CLIENT_ID,
    CLIENT_SECRET,
    browser_callback=True,
)

In [7]:
BASE_URL = "https://fantasysports.yahooapis.com/fantasy/v2"

In [8]:
def api_request(url):
    if not oauth.token_is_valid():
        oauth.refresh_access_token()
    headers = {
        "Authorization": f"Bearer {oauth.access_token}",
        "Content-Type": "application/json"
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(response.text)
        response.raise_for_status()
    xml = response.text
    data = xml_to_dict(response.text)["fantasy_content"]
    return data

In [9]:
def get_league():
    league_key = f"{game_id}.l.{league_id}"
    url = f"{BASE_URL}/league/{league_key};out=standings"
    resp_data = api_request(url)["league"]
    return resp_data

In [10]:
league = get_league()
teams = league["standings"]["teams"]
teams

[{'team_key': '427.l.97108.t.2',
  'team_id': '2',
  'name': 'Eric’s a Bitch Team',
  'url': 'https://hockey.fantasysports.yahoo.com/2023/hockey/97108/2',
  'team_logos': {'team_logo': {'size': 'large',
    'url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_12_k.png'}},
  'waiver_priority': '4',
  'number_of_moves': '59',
  'number_of_trades': '0',
  'roster_adds': {'coverage_type': 'week',
   'coverage_value': '26',
   'value': '0'},
  'clinched_playoffs': '1',
  'league_scoring_type': 'headpoint',
  'draft_position': '4',
  'has_draft_grade': '0',
  'managers': {'manager': {'manager_id': '2',
    'nickname': 'Ken',
    'guid': 'TPMH3J2R4LWWFJV4GPPLVJPZYI',
    'email': 'dragonkenkj@gmail.com',
    'image_url': 'https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg',
    'felo_score': '656',
    'felo_tier': 'silver'}},
  'team_stats': {'coverage_type': 'season',
   'season': '2023',
   'stats': [{'stat_id': '1', 'value': '304'},
    {'stat_id': '2', 'value': '478'},
    {'s

In [11]:
def get_team_name(team_key):
    team_name = next((team["name"] for team in teams if team["team_key"] == team_key), 'Unknown Team')
    return team_name 

In [12]:
LEAGUE_KEY = query.get_league_key(season=None)

In [13]:
league_info = query.get_league_info()

In [14]:
def get_players(player_keys):
    return [
        player 
        for i in range(int(len(player_keys) / 25) + 1)
        for player in query.query(
            f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/players;player_keys={','.join(player_keys[i*25:min((i+1)*25, len(player_keys))])};start={i*25}/stats",
            ["league", "players"]
        )
    ]

In [15]:
team_info = query.get_league_teams()
team_info[3]

Team({
  "draft_position": 2,
  "has_draft_grade": 0,
  "league_scoring_type": "headpoint",
  "managers": {
    "manager": {
      "email": "andrewp3@shaw.ca",
      "felo_score": 530,
      "felo_tier": "bronze",
      "guid": "OUL766QQDYAPQNVXMSEDVLET24",
      "image_url": "https://s.yimg.com/ag/images/8bb55647-c4ee-4c08-9bc5-fa5609b44ae3_64sq.jpg",
      "manager_id": 4,
      "nickname": "Andrew"
    }
  },
  "name": "Miami Steamrollers",
  "number_of_moves": 10,
  "number_of_trades": 0,
  "roster_adds": {
    "coverage_type": "week",
    "coverage_value": 26,
    "value": 0
  },
  "team_id": 4,
  "team_key": "427.l.97108.t.4",
  "team_logos": {
    "team_logo": {
      "size": "large",
      "url": "https://yahoofantasysports-res.cloudinary.com/image/upload/t_s192sq/fantasy-logos/487f5795c5a24fc01ad380d9da51fb26e062fd152c0060b938512522089524b1.png"
    }
  },
  "url": "https://hockey.fantasysports.yahoo.com/2023/hockey/97108/4",
  "waiver_priority": 6
})

In [16]:
def get_full_scoring_df():
    league_settings = query.get_league_settings()
    scoring_categories = league_settings.stat_categories.stats
    for i in range(len(scoring_categories)):
        scoring_categories[i] = scoring_categories[i].clean_data_dict()
    scoring_categories = pd.DataFrame(data=scoring_categories)

    scoring_modifiers = league_settings.stat_modifiers.stats
    scoring_modifiers = [scoring_modifiers[i].clean_data_dict() for i in range(len(scoring_modifiers))]

    scoring_settings = pd.merge(left=scoring_categories, right=pd.DataFrame(data=scoring_modifiers), on='stat_id', how='inner')
    return scoring_settings


def get_scoring_dict_by_stat_id(scoring_settings_df):
    return {str(scoring_settings_df['stat_id'][i]) : float(scoring_settings_df['value'][i]) for i in range(len(scoring_settings_df))}

In [17]:
# LEAGUE_SETTINGS = query.get_league_settings()
# SCORING_CATEGORIES = LEAGUE_SETTINGS.stat_categories.stats
# for i in range(len(SCORING_CATEGORIES)):
#     SCORING_CATEGORIES[i] = SCORING_CATEGORIES[i].clean_data_dict()
# SCORING_CATEGORIES = pd.DataFrame(data=SCORING_CATEGORIES)

# SCORING_MODIFIERS = LEAGUE_SETTINGS.stat_modifiers.stats
# SCORING_MODIFIERS = [SCORING_MODIFIERS[i].clean_data_dict() for i in range(len(SCORING_MODIFIERS))]

# SCORING_SETTINGS = pd.merge(left=SCORING_CATEGORIES, right=pd.DataFrame(data=SCORING_MODIFIERS), on='stat_id', how='inner')

In [18]:
SCORING_SETTINGS = get_full_scoring_df()
SCORING_MODIFIERS_BY_STAT_ID = get_scoring_dict_by_stat_id(SCORING_SETTINGS)

Draft Info

In [19]:
full_draft = query.get_league_draft_results()

Seperate Draft into by Team

In [20]:
full_draft = pd.DataFrame(data=full_draft, columns=['Overall', 'Round', 'Team ID', 'Player ID'])

team_drafts = {}
team_keys = []
for ID in range(1, full_draft['Team ID'].nunique()+1):
    team_keys = [LEAGUE_KEY + '.t.' + str(ID) for ID in range(1, full_draft['Team ID'].nunique()+1)] 
team_keys = tuple(team_keys)

In [21]:
team_drafts = {team_key : full_draft[full_draft['Team ID'] == team_key] for team_key in team_keys}

Assign Best and Worst Draft

In [22]:
'''
Best/Worst Draft
'''
total_draft_points = dict.fromkeys(team_keys, 0)
for team in team_drafts:
    # pull each team draft 
    draft = list(team_drafts[team]['Player ID'])
    stats = get_players(player_keys=draft)
    
    for j in range(len(draft)):
        # pull each player from the draft
        # pull each player's stats
        player_stats = stats[j].player_stats.clean_data_dict()['stats']
        
        for i in range(len(player_stats)):
            # pull league scoring settings
            # multiply league settings by player's stats
            stat = player_stats[i]['stat'].clean_data_dict()
            if str(stat['stat_id']) in list(SCORING_MODIFIERS_BY_STAT_ID.keys()):
                total_draft_points[team] += stat['value'] * SCORING_MODIFIERS_BY_STAT_ID[str(stat['stat_id'])] #list(SCORING_SETTINGS['value'])[list(SCORING_SETTINGS['stat_id']).index(stat['stat_id'])]

total_draft_points = {key : round(value, 1) for key, value in total_draft_points.items()}
total_draft_points = dict(sorted(total_draft_points.items(), key=lambda item: item[1], reverse=True))
total_draft_points = {key : (get_team_name(key), value) for key, value in total_draft_points.items()}

print('Best Drafts:')
print(f'1. {list(total_draft_points.values())[0][0]} ({str(list(total_draft_points.values())[0][1])})')
print(f'2. {list(total_draft_points.values())[1][0]} ({str(list(total_draft_points.values())[1][1])})')
print(f'3. {list(total_draft_points.values())[2][0]} ({str(list(total_draft_points.values())[2][1])})\n')
print('Worst Drafts:')
print(f'1. {list(total_draft_points.values())[len(total_draft_points)-1][0]} ({str(list(total_draft_points.values())[len(total_draft_points)-1][1])})')
print(f'2. {list(total_draft_points.values())[len(total_draft_points)-2][0]} ({str(list(total_draft_points.values())[len(total_draft_points)-2][1])})')
print(f'3. {list(total_draft_points.values())[len(total_draft_points)-3][0]} ({str(list(total_draft_points.values())[len(total_draft_points)-3][1])})')

Best Drafts:
1. Tim's Terrific Team (5554.2)
2. Theodore's Poo Poo Hospital (5315.4)
3. Eric’s a Bitch Team (5312.0)

Worst Drafts:
1. Miami Steamrollers (4597.7)
2. Kevin's Incredible Team (4675.8)
3. Cyrus's Cursed Team (4866.1)


Mike Sillinger Award - Most Picked up/Dropped Player

In [28]:
'''
Mike Sillinger Award
Evaluated by who was dropped the most
Prints Top 10
'''
transactions = query.get_league_transactions()

drops = {}
missed = 0
for transaction in transactions:
    if transaction.type == 'drop' and transaction.status == 'successful':
        try:
            drops[transaction.players[0].player_key][1] += 1
        except:
            drops[transaction.players[0].player_key] = [transaction.players[0].full_name, 1]
    elif transaction.type == 'add/drop' and transaction.status == 'successful':
        try:
            drops[transaction.players[1].player_key][1] += 1
        except:
            drops[transaction.players[1].player_key] = [transaction.players[1].full_name, 1]

drops = dict(sorted(drops.items(), key=lambda item: item[1][1], reverse=True))

top_x = 10
print('The Most Picked-Up/Dropped Players are...')
for i in range(top_x):
    print(f'{i+1}. {list(drops.values())[i][0]} ({str(list(drops.values())[i][1])})')

The Most Picked-Up/Dropped Players are...
1. Matt Boldy (7)
2. Ukko-Pekka Luukkonen (6)
3. Darnell Nurse (5)
4. Jakob Chychrun (5)
5. Jacob Markstrom (5)
6. Brent Burns (5)
7. Claude Giroux (5)
8. Tyler Toffoli (4)
9. Ilya Samsonov (4)
10. Drake Batherson (4)


Closest Weekly Matchups

Playground

In [26]:
drops.values()

dict_values([['Matt Boldy', 7], ['Ukko-Pekka Luukkonen', 6], ['Darnell Nurse', 5], ['Jakob Chychrun', 5], ['Jacob Markstrom', 5], ['Brent Burns', 5], ['Claude Giroux', 5], ['Tyler Toffoli', 4], ['Ilya Samsonov', 4], ['Drake Batherson', 4], ['Josh Manson', 4], ['Connor Bedard', 4], ['Logan Thompson', 4], ['Jeff Skinner', 4], ['Tom Wilson', 3], ['Filip Gustavsson', 3], ['Joey Daccord', 3], ['Boone Jenner', 3], ['Alex Lyon', 3], ['Joseph Woll', 3], ['Antti Raanta', 3], ['Jared McCann', 3], ['Elias Lindholm', 3], ['Laurent Brossoit', 3], ['Adam Larsson', 3], ['Rickard Rakell', 3], ['Brayden McNabb', 3], ['Mattias Ekholm', 3], ['William Karlsson', 3], ['Jonathan Quick', 3], ['Carter Hart', 3], ['Owen Tippett', 3], ['Bowen Byram', 2], ['Jake McCabe', 2], ['Thatcher Demko', 2], ['Scott Wedgewood', 2], ['Mark Scheifele', 2], ['Jonas Johansson', 2], ['Jeremy Swayman', 2], ['Gustav Forsling', 2], ['Karel Vejmelka', 2], ['David Rittich', 2], ['Neal Pionk', 2], ['Seth Jarvis', 2], ['Frederik Ander

In [27]:
transaction_type = []
for transaction in transactions:
    transaction_type.append(transaction.type)

set(transaction_type)

{'add', 'add/drop', 'commish', 'drop', 'trade'}